# DIGIT RECOGNNITION

In [7]:
import numpy as np
import os 
# Step 1: We'll download the database of images from MNIST website - this is maintained
# by a famous Neural network researcher named Yann Lecun
def load_dataset():
    def download(filename, source='http://yann.lecun.com/exdb/mnist/'):
        print ("Downloading ",filename)
        import urllib.request
        urllib.request.urlretrieve(source+filename,filename)
    # This will download the specified file from Yann Lecun's website and store it 
    # on our local disk
    
    import gzip
    
    def load_mnist_images(filename):
        if not os.path.exists(filename):
            download(filename)
        # Checks if the specified file is already there on our local disk
        # if not it will download the file 
        with gzip.open(filename,'rb') as f:
            # Open the zip file of images 
            data=np.frombuffer(f.read(), np.uint8, offset=16)
            # This is some boilerplate to extract data from the zip file
            # This data has 2 issues : its in the form of a 1 d array
            # We have to take this array and convert it into images 
            # Each image has 28x28 pixels , its a monochrome image ie only 1 channel (if
            # it were full-color it would have 3/4 channels R,G,B etc)
            
            # data is currently a numpy array which we we want to reshape into 
            # an array of 28x28 images 
            data=data.reshape(-1,1,28,28)
            # The first dimension is the number of images , by making this -1
            # The number of images will be inferred from the value of the other dimensions
            # and the length of the input array
            
            # The second dimension is the number of channels - here this is 1
            
            # The third and fourth dimensions are the size of the image 28x28
            
            # its in the form of bytes 
            return data/np.float32(256)
        # This will convert the byte value to a float32 in the range [0,1]
        
    def load_mnist_labels(filename):
        if not os.path.exists(filename):
            download(filename)
                # Read the labels which are in a binary file again 
        with gzip.open(filename,'rb') as f:
            data = np.frombuffer(f.read(),np.uint8,offset=8)
                    # This gives a numpy array of integers, the digit value corresponding 
                    # to the images we got earlier 
        return data
    # We can now download and read the training and test data sets - both the images 
    # and their labels 
    
    X_train = load_mnist_images('train-images-idx3-ubyte.gz')
    y_train = load_mnist_labels('train-labels-idx1-ubyte.gz')
    X_test = load_mnist_images('t10k-images-idx3-ubyte.gz')
    y_test = load_mnist_labels('t10k-labels-idx1-ubyte.gz')
    
    return X_train, y_train, X_test, y_test


# In[2]:

X_train, y_train, X_test, y_test = load_dataset()


In [8]:


import matplotlib
matplotlib.use('TkAgg')

import matplotlib.pyplot as plt
plt.show(plt.imshow(X_train[1][0]))

C:\Users\Mahima\anaconda3\lib\site-packages\ipykernel_launcher.py:5: MatplotlibDeprecationWarning: Passing the block parameter of show() positionally is deprecated since Matplotlib 3.1; the parameter will become keyword-only in 3.3.
  """


In [9]:
import lasagne
import theano
import theano.tensor as T



def build_NN(input_var=None):
    l_in = lasagne.layers.InputLayer(shape=(None,1,28,28), input_var=input_var)
    
    l_in_drop = lasagne.layers.DropoutLayer(l_in, p=0.2)
    
    l_hid1=lasagne.layers.DenseLayer(l_in_drop,num_units=800,nonlinearity=lasagne.nonlinearities.rectify,W=lasagne.init.GlorotUniform())
    
    l_hid1_drop=lasagne.layers.DropoutLayer(l_hid1,p=0.5)
    
    l_hid2=lasagne.layers.DenseLayer(l_hid1_drop,num_units=800,nonlinearity=lasagne.nonlinearities.rectify,W=lasagne.init.GlorotUniform())
    
    l_hid2_drop=lasagne.layers.DropoutLayer(l_hid2,p=0.5)
    
    #output layer
    
    l_out = lasagne.layers.DenseLayer(l_hid2_drop, num_units=10, nonlinearity=lasagne.nonlinearities.softmax)
    
    
    return l_out
    

    
input_var = T.tensor4('inputs') #An empty 4 dimensional array
target_var =T.ivector('targets') #An empty 1 dimensionl integer array to represent the labels

network=build_NN(input_var)

prediction = lasagne.layers.get_output(network)
loss = lasagne.objectives.categorical_crossentropy(prediction,target_var)

loss = loss.mean()

params = lasagne.layers.get_all_params(network, trainable=True)
updates = lasagne.updates.nesterov_momentum(loss, params, learning_rate=0.01, momentum=0.9)

train_fn = theano.function([input_var, target_var], loss, updates=updates)
    

In [11]:
num_training_steps = 10

for step in range(num_training_steps):
    train_err=train_fn(X_train, y_train)
    print("Current step is"+ str(step))

Current step is0
Current step is1
Current step is2
Current step is3
Current step is4
Current step is5
Current step is6
Current step is7
Current step is8
Current step is9


In [12]:
test_prediction = lasagne.layers.get_output(network)
val_fn = theano.function([input_var], test_prediction)
val_fn([X_test[0]])


#digit predicted is 7

array([[0.09030878, 0.06858369, 0.10201753, 0.1655319 , 0.07774986,
        0.07323589, 0.11866109, 0.16443439, 0.05329987, 0.08617701]])

In [13]:
y_test[0]

7

In [16]:
test_prediction = lasagne.layers.get_output(network, deterministic=True)
test_acc = T.mean(T.eq(T.argmax(test_prediction, axis=1),target_var),dtype=theano.config.floatX)

acc_fn = theano.function([input_var,target_var], test_acc)

acc_fn(X_test,y_test)


array(0.5203)